In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

In [ ]:
path = Path("../../results/distances/20220807-164740.json")
with path.open() as f:
    results = json.load(f)

In [ ]:
from cProfile import label


parameter = "phi"
data = [(v["parameters"]["cx"]["params"][parameter],v["mean_distance"]) for v in results.values()]
data = sorted(data)


min_x,min_y = zip(min(data, key=lambda tup: tup[1]))
x,y = zip(*data)

plt.figure(figsize=(10,10))
plt.plot(x,y)
plt.plot(min_x, min_y, marker="o", markersize=10, label=f"minimum {min_y[0]} at phi={min_x[0]}")
plt.xlabel(parameter)
plt.ylabel("Mean closest distance")
plt.legend()
